In [27]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
#pd.set_option('display.max_colwidth', -1)  # or 199


# reinigen von leeren zeilen
# reinigen von zeilen ohne nutzbaren wert
# formatieren zum richtigen pandas oder numpy format


df = pd.read_csv('cleanednkm_open_F1014.csv',index_col=0)

print(len(df))
#subset all rows where the first colum is not null and exlcude them with ~
df = df[~df.iloc[:,0].isnull()]

print(len(df))


df0 = df.iloc[:,0:2]
df0.head()




10556
10556


,F1014anonym,F1014C1
2722,Nichts,NaN
2723,Nichts,NaN
2724,Nichts,NaN
2725,Nichts,NaN
2726,Nichts,NaN


In [ ]:
multilabel = False

if multilabel:
    bool_list = ~df.isnull().values.all(axis=0)
    table_width =  max([i for i, val in enumerate(bool_list) if val])
    print(bool_list, table_width)


    #drop sparse columns
    df = df.iloc[:,0:table_width+1]
    df.head()

    df0 = df.iloc[:,0:2]
    df0 = df0.rename(columns={'F1014anonym':'F1014anonym' , 'F1014C1':'label'})
    print(df0.shape, df0.columns)
    ser = df.iloc[:,0].to_frame()


    for i in range(2,table_width):
        temp_df = pd.DataFrame()
        temp_df = ser.join(df.iloc[:,i])
        answer, label = temp_df.columns
        temp_df = temp_df.rename(columns={answer:'F1014anonym', label:'label'})
        print('Tmp ',temp_df.shape)
        df0 = df0.append(temp_df, ignore_index=True).reset_index(drop=True)
    print('shape ', df0.shape)
    df0.isnull().values.all(axis=0)

else:
    
    df0 = df0.rename(columns={'F1014anonym':'F1014anonym' , 'F1014C1':'label'})

df0 = df0.replace(np.nan, 'Enthaltung')
df0.iloc[:,1].unique()
df1 = df0
df1.head()



In [21]:
# filtering on answer fields

df1['F1014anonym'] = df1['F1014anonym'].replace([r'^NICHTS$',r'^nichts$',r'^Nichts$'], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^NICHTS\.$',r'^nichts\.$',r'^Nichts\.$'], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^Nichts $',r'^NICHTS $', r'^nichts $' ], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^NICHTS!$',r'^nichts!$',r'^Nichts!$'], 'Nichts', regex=True )

df1['F1014anonym'] = df1['F1014anonym'].replace([r'^NIX$',r'^nix$',r'^Nix$'], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^NIX\.$',r'^nix\.$',r'^Nix\.$'], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^Nix $',r'^NIX $', r'^nix $' ], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^Nix!$',r'^NIX!$', r'^nix!$' ], 'Nichts', regex=True )


df1['F1014anonym'] = df1['F1014anonym'].replace([r'^.*Nicht+.*$' ,r'^.*nicht+.*$', r'^.*nicht+.*$', r'^.*Nichts+.*$' ], 'Nichts', regex=True )
df1['F1014anonym'] = df1['F1014anonym'].replace([r'^.*Nichts+.*$' ,r'^.*NICHTS+.*$', r'^.*nichts+.*$', r'^.*Nichts+.*$' ], 'Nichts', regex=True )
#df1['F1014anonym'] = df1['F1014anonym'].replace([r'^-+$', r'^.* .*$', r'^.* .*$', r'^\.$', r'^.+$', r'^—+$', r'^\?+$', r'^X+$', r'^x+$'], 'keine Angabe', regex=True )

df1['F1014anonym'] = df1['F1014anonym'].replace([r'^nein$', r'^NEIN* .*$', r'^Nein$'], 'Nichts', regex=True )



df1['F1014anonym'] = df1['F1014anonym'].replace([r'-{3,}',r'-([\w\s]*?)-', r'(..\.)',r'([xX]{3})'], 'Nichts', regex=True) #r'-\s(\w+|\s+)*-'

#Zum Anonymisieren von Namen in Kundenantworten
df1['F1014anonym'] = df1['F1014anonym'].replace(r'[Ff]rau\s\S+', 'Frau', regex=True)
df1['F1014anonym'] = df1['F1014anonym'].replace(r'[Hh]err\s\S+', 'Herr', regex=True)




'''
Further grouping of labels to reduce classes

Fördermittel (zu schnell) aufgebraucht                              8
spontane Änderung der Förderungs- bzw. Vertragsbedingungen          5
mangelhafter Support                                                4
Bereitstellungszinsen / Bereitstellungszeit                         1
Rechnungsnachweise erforderlich                                     1

diese zu sonstiges zuweisen, da die klassen sonst nicht ausgewogen genug sind
'''
df1['label'] = df1['label'].replace('Fördermittel (zu schnell) aufgebraucht', 'Sonstiges',regex=True)
df1['label'] = df1['label'].replace(r'spontane Änderung der Förderungs- bzw. Vertragsbedingungen', 'Sonstiges', regex=True)
df1['label'] = df1['label'].replace(r'mangelhafter Support', 'Sonstiges', regex=True)
df1['label'] = df1['label'].replace(r'Bereitstellungszinsen / Bereitstellungszeit', 'Sonstiges', regex=True)
df1['label'] = df1['label'].replace(r'Rechnungsnachweise erforderlich', 'Sonstiges', regex=True)

df1['label'] = df1['label'].replace('Zinsen zu hoch/ keine marktüblichen Zinsen', 'Zinsen',regex=True)
df1['label'] = df1['label'].replace(r'Der derzeit niedrige Zinssatz', 'Zinsen', regex=True)
df1['label'] = df1['label'].replace(r'zu kurze Zinsbindungsfristen', 'Zinsen', regex=True)
df1['label'] = df1['label'].replace(r'Online-Verfahren', 'Aufwändige Beantragung', regex=True)


df1.head()



#Ende Datenbereinigung



'\ntodo aus den tags dummies machen ( maximal 4 klassen)\n\nFördermittel (zu schnell) aufgebraucht                              8\nspontane Änderung der Förderungs- bzw. Vertragsbedingungen          5\nmangelhafter Support                                                4\nBereitstellungszinsen / Bereitstellungszeit                         1\nRechnungsnachweise erforderlich                                     1\n\ndiese zu sonstiges zuweisen, da die klassen sonst nicht ausgewogen genug sind\n'

,F1014anonym,label
2722,Nichts,Enthaltung
2723,Nichts,Enthaltung
2724,Nichts,Enthaltung
2725,Nichts,Enthaltung
2726,Nichts,Enthaltung


In [28]:
# Preprocessing of the labels making them categorical
# a temporary list to store the string labels
temp_list = df1.iloc[:,1].unique()

cleaned_list = [x for x in temp_list if str(x) != 'nan']

cleaned_list = [x for x in temp_list if x != 'nan']

# dictionary that maps integer to its string value 
label_dict = {}

# list to store integer labels 
int_labels = []

for i in range(len(cleaned_list )):
    label_dict[i] = cleaned_list [i]
    int_labels.append(i)


print(cleaned_list )
print(int_labels)
print(label_dict)

print(df1.head)

['Enthaltung', 'Nichts', 'kein / unbekannter Bedarf (bei Bekannten / Gesprächspartnern)', 'Unklare Entscheidungsverfahren', 'Lange Bearbeitungsdauer', 'Sonstiges', 'Höhe des Zuschusses nicht ausreichend', 'unflexible Bedingungen', 'Aufwändige Beantragung', 'Kosten-/Nutzenverhältnis', 'Erreichbarkeit des Angebots (Serverausfall)', 'Abhängigkeit von der Bank', 'Zinsen', 'Kosten für Energieberater', 'Zusätzliche Kosten bei Ausführung/ für Ingenieurleistungen', 'Sondertilgungen nicht möglich', 'Fördermittel (zu schnell) aufgebraucht']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{0: 'Enthaltung', 1: 'Nichts', 2: 'kein / unbekannter Bedarf (bei Bekannten / Gesprächspartnern)', 3: 'Unklare Entscheidungsverfahren', 4: 'Lange Bearbeitungsdauer', 5: 'Sonstiges', 6: 'Höhe des Zuschusses nicht ausreichend', 7: 'unflexible Bedingungen', 8: 'Aufwändige Beantragung', 9: 'Kosten-/Nutzenverhältnis', 10: 'Erreichbarkeit des Angebots (Serverausfall)', 11: 'Abhängigkeit von der Bank', 12: '

In [29]:
inv_map = {v: k for k, v in label_dict.items()}
df_cat = df1
df_cat['cat labels'] = df1.iloc[:,1].map(inv_map)

df_cat

,F1014anonym,label,cat labels
2722,Nichts,Enthaltung,0
2723,Nichts,Enthaltung,0
2724,Nichts,Enthaltung,0
2725,Nichts,Enthaltung,0
2726,Nichts,Enthaltung,0
2727,Nichts,Enthaltung,0
2728,Nichts,Enthaltung,0
2729,Nichts,Enthaltung,0
2730,Nichts,Enthaltung,0
2732,Nichts,Enthaltung,0


In [30]:
df_cat.to_csv('one_categorised_labels_nkm.csv')

In [6]:
df_cat.head()

,F1014anonym,label,cat labels
0,Nichts,Enthaltung,0
1,Nichts,Enthaltung,0
2,Nichts,Enthaltung,0
3,Nichts,Enthaltung,0
4,Nichts,Enthaltung,0
